In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('FY0.csv')

In [ ]:
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

numeric_cols = ['ESG-FY0', 'SP-FY0', 'GP-FY0', 'EP-FY0', 'Tobin Q-FY0', 
                'Growth-FY0', 'CashFlow-FY0', 'Liquidity-FY0', 
                'Total Assets Reported-FY0', 'Profitability-FY0', 'Debt-FY0']

data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Handling missing values by replacing them with mean
data.fillna(data.mean(), inplace=True)

# Checking for missing values and infinite values after handling
missing_values_after = data.isnull().sum()
infinite_values_after = data.replace([np.inf, -np.inf], np.nan).isnull().sum()

# Preparing the data for VIF calculation again
vif_data_cleaned = data[numeric_cols]
vif_data_cleaned = add_constant(vif_data_cleaned)  # Adding a constant for the VIF calculation

# Calculating VIF for each variable again
vif_cleaned = pd.DataFrame()
vif_cleaned["Variable"] = vif_data_cleaned.columns
vif_cleaned["VIF"] = [variance_inflation_factor(vif_data_cleaned.values, i) for i in range(vif_data_cleaned.shape[1])]

missing_values_after, infinite_values_after, vif_cleaned


In [ ]:
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

# Preparing the data for regression analysis
X = data[['ESG-FY0', 'SP-FY0', 'GP-FY0', 'EP-FY0', 'Growth-FY0', 'CashFlow-FY0', 'Liquidity-FY0', 
          'Total Assets Reported-FY0', 'Profitability-FY0', 'Debt-FY0']]
y = data['Tobin Q-FY0']

# Adding a constant to the model
X = sm.add_constant(X)

# Running the regression analysis
model = sm.OLS(y, X).fit()

# Performing the Breusch-Pagan test for heteroskedasticity
bp_test = het_breuschpagan(model.resid, model.model.exog)

# Regression results and Breusch-Pagan test results
model_summary = model.summary()
bp_test_results = {'Lagrange Multiplier Statistic': bp_test[0], 'p-value': bp_test[1], 
                   'f-value': bp_test[2], 'f p-value': bp_test[3]}

model_summary, bp_test_results


In [ ]:
selected_columns = [
    'ESG-FY0',  # ESG scores
    'SP-FY0', 'GP-FY0', 'EP-FY0',  # Financial metrics
    'Growth-FY0', 'CashFlow-FY0', 'Liquidity-FY0',  # Growth and liquidity metrics
    'Total Assets Reported-FY0', 'Profitability-FY0', 'Debt-FY0',  # Other financial variables
    'Tobin Q-FY0'  # Dependent variable
]

# Filtering the dataset
filtered_data = data[selected_columns]

# Handling missing values
filtered_data.fillna(filtered_data.mean(), inplace=True)

# Defining the independent variables (X) and the dependent variable (y)
X = filtered_data.drop('Tobin Q-FY0', axis=1)
y = filtered_data['Tobin Q-FY0']

# Adding a constant to the model
X = sm.add_constant(X)

# Running the regression analysis
model = sm.OLS(y, X).fit()

# Getting the summary of the regression analysis
model_summary = model.summary()
model_summary

In [ ]:
import pandas as pd

# Creating a DataFrame with the provided variables
data = {
    "Constituent variable Name": [
        "Identifier",
        "Company Name",
        "ESG Score",
        "Environment Pillar Score",
        "Social Pillar Score",
        "Governance Pillar Score",
        "Weighted Average Cost of Capital (WACC)",
        "Market Capitalization",
        "Bid and Ask Prices",
        "Cash Flow",
        "Total Liabilities",
        "Total Common Outstanding Shares",
        "Closed Price",
        "Total Assets Reported",
        "Net Income Before Extraordinary Items",
        "Long-Term Debt",
        "Return on Asset (ROA)",
        "Tangible Book Value",
        "Total Debt",
        "Outstanding Shares"
    ],
    "Description": [
        "Unique alphanumeric codes assigned to each company for identification purposes.",
        "Official registered names of the corporations.",
        "Aggregate scores evaluating companies' overall ESG performance.",
        "Assessment of companies' environmental stewardship and impact.",
        "Evaluation of companies' social responsibility efforts and impacts.",
        "Ratings reflecting the quality of companies' governance structures and practices.",
        "Calculation of companies' average cost of capital, accounting for the proportion of each capital type.",
        "Valuation of companies based on the total market value of their outstanding shares.",
        "The highest price a buyer is willing to pay (bid) and the lowest price a seller is willing to accept (ask) for a stock.",
        "Detailed financial statements outlining the inflows and outflows of cash.",
        "Comprehensive enumeration of companies' financial obligations.",
        "The total number of shares currently held by all shareholders.",
        "The price at which a stock last traded during a regular trading session.",
        "Complete listing of companies' assets.",
        "Companies' earnings excluding one-time events.",
        "Debt obligations with a maturity of more than one year.",
        "A measure of how effectively a company uses its assets to generate earnings.",
        "The net asset value of a company, excluding intangible assets.",
        "The sum of a company's short and long-term liabilities.",
        "Detailed data collected for each of the six fiscal years under consideration."
    ]
}

df = pd.DataFrame(data)
df.style.set_properties(**{'text-align':'left'}).hide_index()


In [ ]:
import pandas as pd

control_variables_data = {
    "Variable Name": [
        "Growth",
        "Profitability",
        "Liquidity",
        "Cashflow",
        "Debt",
        "Total Assets",
        "Leverage"
    ],
    "Calculation": [
        "Log10 of the market capitalization",
        "Net income before extraordinary items/Total assets.",
        "Log10 of the average bid-ask spread percentage.",
        "Log10 of the cash flow.",
        "Log10 of the long-term interest-bearing debt.",
        "Log10 of the total assets",
        "Total debt/Tangible_Book_Value"
    ]
}

control_variables_df = pd.DataFrame(control_variables_data)
control_variables_df.style.set_properties(**{'text-align': 'right'}).hide_index()
